# Ingest ADS Notebook Files (Project)

![files_to_database](files_to_database.png)

This Notebook will load selected files (from a local or networked file system) into a SQL Server database.

> For best results, use this notebook to create an **eltSnap** project

## Reset staging table

In [1]:
Invoke-Sqlcmd -ServerInstance <#${eltsnap_v2_Server}#>'localhost'<##> -Database <#${eltsnap_v2_Database}#>'eltsnap_v2'<##> -Query 'TRUNCATE TABLE [stg].[files]'

## Load Files into table: [stg].[files]


In [2]:
Get-ChildItem -Path <#${notebook_repos}#>'C:\ADS\Repository\'<##> -Recurse -Include *.ipynb | Select-Object @{l='FullName';e={($_.FullName).Replace(<#${notebook_repos}#>'C:\ADS\Repository\'<##>,'')}}, BaseName, @{l='Extension';e={($_.Extension).Replace('.','')}}, LastWriteTime, Length, @{l='Contents';e={(Get-Content $_ -Raw).ToString()}} | Write-SqlTableData -ServerInstance <#${eltsnap_v2_Server}#>'localhost'<##> -DatabaseName <#${eltsnap_v2_Database}#>'eltsnap_v2'<##> -SchemaName 'stg' -TableName 'files'

## Merge table: [stg].[files] into: [ads].[files]

In [3]:
Invoke-Sqlcmd -ServerInstance <#${eltsnap_v2_Server}#>'localhost'<##> -Database <#${eltsnap_v2_Database}#>'eltsnap_v2'<##> -Query "EXEC [ads].[Merge Staged Files];"


row_count
---------
        9




## Ingest all Notebooks Files into table: [ads].[notebooks]

In [4]:
Invoke-Sqlcmd -ServerInstance <#${eltsnap_v2_Server}#>'localhost'<##> -Database <#${eltsnap_v2_Database}#>'eltsnap_v2'<##> -Query "EXEC [ads].[Ingest All Notebooks];"